In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [26]:
train_data = pd.read_csv('train.csv',index_col='SK_ID_CURR').sort_index(ascending=True)

In [27]:
train_data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,AGE,YEARS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_TOTAL_DAY
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
0,0,Cash loans,M,0,202500.0,44487.0,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.026392,32,1,NaN,1.0,Other,0.284570,0.285180,0.000000
1,1,Cash loans,F,0,175500.0,26316.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.003122,52,25,Accountants,2.0,School,NaN,NaN,1095.000000
2,0,Cash loans,F,1,58500.0,11263.5,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.031329,39,10,NaN,3.0,School,0.439261,0.415347,90.000000
3,0,Cash loans,F,2,54000.0,20547.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.004849,43,16,Cleaning staff,4.0,Business Entity Type 2,NaN,0.397946,940.000000
4,0,Cash loans,F,0,135000.0,25033.5,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.018801,48,1,Medicine staff,2.0,University,NaN,0.529890,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307505,0,Cash loans,F,1,157500.0,42790.5,"Spouse, partner",Commercial associate,Higher education,Married,House / apartment,0.046220,42,20,Laborers,3.0,Transport: type 2,NaN,NaN,730.041667
307506,0,Cash loans,F,0,67500.0,9940.5,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.020246,60,-1000,NaN,1.0,XNA,NaN,0.701696,365.000000
307507,0,Revolving loans,F,0,450000.0,61875.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.032561,29,3,Managers,2.0,Business Entity Type 3,NaN,NaN,NaN


In [28]:
x_train = train_data.drop(columns='TARGET')
y_train = train_data['TARGET'].astype('float')

In [29]:
numeric_columns = x_train._get_numeric_data().columns
categorical_columns = x_train.columns[~x_train.columns.isin(numeric_columns)]

In [30]:

numeric_transformer = Pipeline(steps= [
        ('imputer', SimpleImputer(strategy='median')),
        ('mmsc', MinMaxScaler(feature_range=(0,1))),

])

categorical_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder='passthrough')

# Create a pipeline with preprocessing and LogisticRegression
pipeline = Pipeline(steps= [
    ('preprocess',preprocessor)
])

# Fit the pipeline on training data
x_train = pd.DataFrame(pipeline.fit_transform(x_train))



In [33]:
x_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_application_test.csv',index_col= 0)
x_test = x_test.set_index('SK_ID_CURR').sort_index(ascending=True)
submit = pd.DataFrame(x_test.index)

# update x_test
x_test['AMT_REQ_CREDIT_BUREAU_TOTAL_DAY'] = (
    x_test['AMT_REQ_CREDIT_BUREAU_HOUR'] / 24 +
    x_test['AMT_REQ_CREDIT_BUREAU_DAY'] +
    x_test['AMT_REQ_CREDIT_BUREAU_MON'] * 30 +
    x_test['AMT_REQ_CREDIT_BUREAU_YEAR'] * 365 +
    x_test['AMT_REQ_CREDIT_BUREAU_WEEK'] * 7 +
    x_test['AMT_REQ_CREDIT_BUREAU_QRT'] * 3 * 30
)
x_test['DAYS_BIRTH'] = x_test['DAYS_BIRTH'].apply(lambda x: int((x * -1) / 365))
x_test['DAYS_EMPLOYED'] = x_test['DAYS_EMPLOYED'].apply(lambda x: int((x * -1) / 365))
x_test = x_test.rename({'DAYS_BIRTH': 'AGE', 'DAYS_EMPLOYED' : 'YEARS_EMPLOYED'}, axis=1)
x_test = x_test[train_data.drop(columns='TARGET').columns]
x_test

,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,AGE,YEARS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_TOTAL_DAY
SK_ID_CURR,,,,,,,,,,,,,,,,,,,
5,Cash loans,F,1,540000.0,62698.5,Unaccompanied,Working,Higher education,Married,House / apartment,0.003122,49,3,Managers,3.0,Other,0.696321,0.475850,30.0
13,Cash loans,F,1,72000.0,23508.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.015221,46,3,Sales staff,3.0,Self-employed,NaN,0.372334,0.0
16,Cash loans,F,0,202500.0,29151.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,Municipal apartment,0.018801,34,6,Sales staff,1.0,Business Entity Type 3,0.695093,0.508287,1095.0
19,Cash loans,M,0,135000.0,16758.0,Family,State servant,Secondary / secondary special,Civil marriage,House / apartment,0.009657,30,7,Core staff,2.0,School,NaN,0.456110,1460.0
25,Cash loans,F,2,90000.0,26217.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.006207,40,8,NaN,4.0,Kindergarten,0.517339,0.504681,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307487,Cash loans,F,0,306000.0,24070.5,Unaccompanied,State servant,Higher education,Civil marriage,House / apartment,0.026392,47,13,High skill tech staff,2.0,University,NaN,0.321735,0.0
307490,Cash loans,M,2,270000.0,46899.0,Family,State servant,Higher education,Married,House / apartment,0.025164,37,9,NaN,4.0,Police,0.657034,0.520898,1125.0
307497,Cash loans,F,0,189000.0,42790.5,Unaccompanied,Commercial associate,Secondary / secondary special,Separated,House / apartment,0.018801,43,3,Laborers,1.0,Business Entity Type 3,NaN,0.812823,365.0


In [34]:
# Predict
model = LogisticRegression(random_state= 0)
model.fit(x_train,y_train)
x_test = pipeline.transform(x_test)
y_pred = model.predict_proba(x_test)[:, 1]


d:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
submit['TARGET'] = y_pred
submit = submit.set_index('SK_ID_CURR')
submit.to_csv('../Submission.csv')
